#### import dependencies

In [1]:
import pandas as pd
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


#### import needed API keys

In [2]:
from config import yelp_api_key_Austin, yelp_api_key_Shane, yelp_client_id_Shane


#### set input CSV data file filepath variable

In [3]:
# dallas_open_data = "../Other/Resources/dallasGoogleRating.csv"
dallas_open_data = "../Other/Resources/Google Lat Long Results_RestBarCafe.csv"


#### set output CSV data file filepath variable

In [4]:
output_data_filepath = "output_data/"

#### import Dallas Open Data CSV data file (with lat/lng added; obtained from Google API) into dataframe & visualize data frame

In [5]:
dallas_open_data_df = pd.read_csv(dallas_open_data, dtype="object", encoding="utf-8")
dallas_open_data_df.head()


,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,A & A MART,Routine,5/14/19,81,309,MARSALIS,NaN,75203,309 S MARSALIS AVE,5/1/19,FY2019,32.7444815,-96.8160463,Restaurant,3.8,717
1,A & J BAKERY,Routine,4/30/19,92,3515,OAK LAWN,NaN,75219,3515 OAK LAWN AVE,4/1/19,FY2019,32.8123799,-96.8054238,Restaurant,4,60
2,A & K MART,Routine,2/7/19,98,5723,SINGLETON,NaN,75212,5723 SINGLETON BLVD,2/1/19,FY2019,32.7789351,-96.9141423,Restaurant,4.3,23
3,A MOTIONS FOOD MART,Routine,2/22/18,80,829,CORINTH ST,NaN,75203,829 S CORINTH ST RD,2/1/18,FY2018,32.7394658,-96.7988631,Restaurant,NaN,NaN
4,A PLUS GROCERY BEER & WINE,Routine,2/14/19,82,4407,MOCKINGBIRD,NaN,75209,4407 W MOCKINGBIRD LN,2/1/19,FY2019,32.836885,-96.830941,Restaurant,4.3,44


#### create a second data frame to use below and visualize the data frame

In [7]:
# df = dallas_open_data_df
df = dallas_open_data_df
df.head()

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,Inspection Year,Lat,Long,Type,Rating,Rating Count
0,A & A MART,Routine,5/14/19,81,309,MARSALIS,NaN,75203,309 S MARSALIS AVE,5/1/19,FY2019,32.7444815,-96.8160463,Restaurant,3.8,717
1,A & J BAKERY,Routine,4/30/19,92,3515,OAK LAWN,NaN,75219,3515 OAK LAWN AVE,4/1/19,FY2019,32.8123799,-96.8054238,Restaurant,4,60
2,A & K MART,Routine,2/7/19,98,5723,SINGLETON,NaN,75212,5723 SINGLETON BLVD,2/1/19,FY2019,32.7789351,-96.9141423,Restaurant,4.3,23
3,A MOTIONS FOOD MART,Routine,2/22/18,80,829,CORINTH ST,NaN,75203,829 S CORINTH ST RD,2/1/18,FY2018,32.7394658,-96.7988631,Restaurant,NaN,NaN
4,A PLUS GROCERY BEER & WINE,Routine,2/14/19,82,4407,MOCKINGBIRD,NaN,75209,4407 W MOCKINGBIRD LN,2/1/19,FY2019,32.836885,-96.830941,Restaurant,4.3,44


#### check how many records are coming through

In [8]:
len(df)

2850

####  set API URL and API key variables to pass through in the API request

In [12]:
api_key_1 = yelp_api_key_Shane
# api_key_2 = yelp_api_key_Austin

headers_1 = {'Authorization': 'Bearer %s' % api_key_1}
# headers_2 = {'Authorization': 'Bearer %s' % api_key_2}

url = 'https://api.yelp.com/v3/businesses/search'


#### create blank/empty variables for the data input into and returned from the API get requests and counters

In [13]:
# # setup of variables (initially as blank); to be updated & used in the for loop with API call for each row
search_name = ""
search_address = ""
search_street_number = ""
search_street_name = ""
search_zip_code = ""
search_lat = ""
search_lng = ""

# setup counters, initially set to 1, to count cities with API calls that successfully retrieve data
attempt_counter = 1
record_counter = 1
set_of_records_counter = 1
record_in_set_counter = 1

# create list to hold all results, in the event we need to reference later
yelp_results_all = []
    

#### loop through rows in df, sending an API request for that restaurant using restaurant_name, street_number, street_name and zip_code

In [14]:
# start log for data retrieval from API Call
print("Beginning Data Retrieval")
print("-----------------------------")

# loop through df results & complete API call for each; append df with data when successful
for index, row in df.iterrows():
    # get restaurant data from df, setting respective variables for that restaurant
    search_name = row["Restaurant Name"]
    search_street_number = row["Street Number"]
    search_street_name = row["Street Name"]
    search_zip_code = row["Zip"]
    search_lat = row["Lat"]
    search_lng = row["Long"]
    
    # set API request parameters
    params = {
        "term": f"{search_name}",
        "location": f"{search_street_number} {search_street_name} Dallas, TX {search_zip_code}",
        "radius": 4000,
        "limit": 5,
        "sort_by": "best_match"
    }
    
    # set yelp_response to results from API request
    yelp_response = requests.get(url, params=params, headers=headers_1).json()
    
    # extend yelp_response_all to hold all results in case we need to use this later
    try:
        yelp_results_all.extend(yelp_response["businesses"])
    except:
        pass
    
    # traverse json object, set yelp_results to only business data for the best match result (0 index position)
    try:
        yelp_results = yelp_response["businesses"][0]
#         yelp_results = yelp_response["businesses"]
        print(f"Attempt: {attempt_counter} | Processing record {record_counter} ({record_in_set_counter} of set {set_of_records_counter}) | {search_name}, {search_street_number} {search_street_name} Dallas, TX {search_zip_code}")

        # not all results have all fields; to prevent error, try/except error handling
        try:
            yelp_category_1 = yelp_results["categories"][0]["title"]
            df.loc[index, "yelp_category_1"] = yelp_category_1
        except:
            pass
            
        try:
            yelp_category_2 = yelp_results["categories"][1]["title"]
            df.loc[index, "yelp_category_2"] = yelp_category_2
        except:
            pass
            
        try:
            yelp_category_3 = yelp_results["categories"][2]["title"]
            df.loc[index, "yelp_category_3"] = yelp_category_3
        except:
            pass
        
        try:
            yelp_latitude = yelp_results["coordinates"]["latitude"]
            df.loc[index, "yelp_lat"] = yelp_latitude
        except:
            pass
        
        try:
            yelp_longitude = yelp_results["coordinates"]["longitude"]
            df.loc[index, "yelp_lng"] = yelp_longitude
        except:
            pass
        
        try:
            yelp_is_closed = yelp_results["is_closed"]
            df.loc[index, "yelp_is_closed"] = yelp_is_closed
        except:
            pass
        
        try:
            yelp_address_1 = yelp_results["location"]["address1"]
            df.loc[index, "yelp_address_1"] = yelp_address_1
        except:
            pass
        
        try:
            yelp_address_2 = yelp_results["location"]["address2"]
            df.loc[index, "yelp_address_2"] = yelp_address_2
        except:
            pass
        
        try:
            yelp_zip_code = yelp_results["location"]["zip_code"]
            df.loc[index, "yelp_zip_code"] = yelp_zip_code
        except:
            pass
        
        try:
            yelp_name = yelp_results["name"]
            df.loc[index, "yelp_name"] = yelp_name
        except:
            pass
        
        try:
            yelp_price = yelp_results["price"]
            df.loc[index, "yelp_price"] = yelp_price
        except:
            pass
        
        try:
            yelp_rating = yelp_results["rating"]
            df.loc[index, "yelp_rating"] = yelp_rating
        except:
            pass
        
        try:
            yelp_review_count = yelp_results["review_count"]
            df.loc[index, "yelp_review_count"] = yelp_review_count
        except:
            pass

        # update counters
        attempt_counter += 1
        record_counter += 1
        
        # for set counters, once 50 is reached: reset record_in_set_counter = 1 and add 1 to set_counter
        if record_in_set_counter >= 50:
            set_of_records_counter += 1
            record_in_set_counter = 1
        else:
            record_in_set_counter += 1

    except:
        print(f"Skipping...  Attempt: {attempt_counter} | Record ({search_name} {search_street_number} {search_street_name} Dallas, TX {search_zip_code}) is not found.")
        # update attempt_counter
        attempt_counter += 1
#         pass

# end log for data retrieval; print "Data Retrieval Complete" when at the end of the loop
print("----------------------------")
print("Data Retrieval Complete")
print("----------------------------")
    

Beginning Data Retrieval
-----------------------------
Attempt: 1 | Processing record 1 (1 of set 1) | A & A MART, 309 MARSALIS, 75203
Attempt: 2 | Processing record 2 (2 of set 1) | A & J BAKERY, 3515 OAK LAWN, 75219
Attempt: 3 | Processing record 3 (3 of set 1) | A & K MART, 5723 SINGLETON, 75212
Attempt: 4 | Processing record 4 (4 of set 1) | A MOTIONS FOOD MART, 829 CORINTH ST, 75203
Attempt: 5 | Processing record 5 (5 of set 1) | A PLUS GROCERY BEER & WINE, 4407 MOCKINGBIRD, 75209
Attempt: 6 | Processing record 6 (6 of set 1) | A SANDWICH SHOP, 8333 DOUGLAS, 75225
Attempt: 7 | Processing record 7 (7 of set 1) | A TASTE OF WAYS CATERING, 2949 KIEST, 75216
Attempt: 8 | Processing record 8 (8 of set 1) | A TO Z FOOD MART, 2120 ST AUGUSTINE, 75227
Attempt: 9 | Processing record 9 (9 of set 1) | A-MOTION FOOD MART, 829 CORINTH, 75203
Attempt: 10 | Processing record 10 (10 of set 1) | A-PLUS EXPRESS KIOSK, 1500 MARILLA, 75201
Attempt: 11 | Processing record 11 (11 of set 1) | A&A PIZZA,

Attempt: 92 | Processing record 73 (23 of set 2) | ASIAN GROCERY, 9191 FOREST, 75243
Attempt: 93 | Processing record 74 (24 of set 2) | ASIAN MINT, 11617 CENTRAL, 75243
Attempt: 94 | Processing record 75 (25 of set 2) | ASIAN STAR BUFFET, 9208 RL THORNTON, 75228
Attempt: 95 | Processing record 76 (26 of set 2) | ASO ROCK RESTAURANT & LOUNGE, 9220 SKILLMAN, 75243
Attempt: 96 | Processing record 77 (27 of set 2) | ATOMIC PIE, 9660 AUDELIA, 75238
Attempt: 97 | Processing record 78 (28 of set 2) | AUNT IRENE'S KITCHEN & CATERING, 3309 MALCOLM X, 75216
Attempt: 98 | Processing record 79 (29 of set 2) | AUNTIE ANNE'S, 13350 DALLAS, 75240
Skipping...  Attempt: 99 | Record (AUTRY'S GROCERY, 4903 LANCASTER, 75216) is not found.
Attempt: 100 | Processing record 80 (30 of set 2) | AVANTI RISTORANTI, 2720 MCKINNEY, 75204
Attempt: 101 | Processing record 81 (31 of set 2) | AVENUE, 2912 MCKINNEY, 75204
Attempt: 102 | Processing record 82 (32 of set 2) | AVILA'S MEXICAN RESTAURANT, 4714 MAPLE, 75219


Attempt: 182 | Processing record 148 (48 of set 3) | BISOUS BISOUS PâTISSERIE, 3700 MCKINNEY, 75204
Attempt: 183 | Processing record 149 (49 of set 3) | BISTRO CAFE&GRILL, 1111 MOCKINGBIRD, 75247
Attempt: 184 | Processing record 150 (50 of set 3) | BISTRO Q, 8300 JOHN W. CARPENTER, 75247
Attempt: 185 | Processing record 151 (1 of set 4) | BITE BISTRO, 1150 EMPIRE CENTRAL, 75247
Skipping...  Attempt: 186 | Record (BK FOOD MART, 8200 RL THORNTON, 75228) is not found.
Skipping...  Attempt: 187 | Record (BKC KITCHEN, 1912 MLK, 75215) is not found.
Attempt: 188 | Processing record 152 (2 of set 4) | BLACK FRIAR PUB UPSTAIRS, 2621 MCKINNEY, 75204
Attempt: 189 | Processing record 153 (3 of set 4) | BLACK JACK PIZZA, 2120 ST AUGUSTINE, 75227
Attempt: 190 | Processing record 154 (4 of set 4) | BLACK SWAN SALOON, 2708 ELM, 75226
Attempt: 191 | Processing record 155 (5 of set 4) | BLU'S, 17630 PRESTON, 75252
Attempt: 192 | Processing record 156 (6 of set 4) | BLUE BIRD, 11434 EMERALD, 75229
Attem

Attempt: 273 | Processing record 227 (27 of set 5) | BURGER KING, 1550 COCKRELL HILL, 75211
Attempt: 274 | Processing record 228 (28 of set 5) | BURGER KING, 4820 ILLINOIS, 75211
Attempt: 275 | Processing record 229 (29 of set 5) | BURGER KING, 5119 ROSS, 75206
Attempt: 276 | Processing record 230 (30 of set 5) | BURGER KING, 7751 BONNIE VIEW, 75241
Attempt: 277 | Processing record 231 (31 of set 5) | BURGER KING, 2828 NORTHWEST, 75220
Attempt: 278 | Processing record 232 (32 of set 5) | BURGER KING, 725 ILLINOIS, 75224
Attempt: 279 | Processing record 233 (33 of set 5) | BURGER KINGS, 2439 LEDBETTER, 75216
Attempt: 280 | Processing record 234 (34 of set 5) | BURGER STREET, 5657 MOCKINGBIRD, 75206
Attempt: 281 | Processing record 235 (35 of set 5) | BURGER STREET, 9180 SKILLMAN, 75243
Attempt: 282 | Processing record 236 (36 of set 5) | BURGUESA, 1201 ELM, 75202
Attempt: 283 | Processing record 237 (37 of set 5) | BURGUESA, 710 FORT WORTH, 75208
Attempt: 284 | Processing record 238 (38

Attempt: 365 | Processing record 302 (2 of set 7) | CEDAR SOCIAL, 1326 LAMAR, 75215
Attempt: 366 | Processing record 303 (3 of set 7) | CEDAR SPRINGS TAP HOUSE, 4123 CEDAR SPRINGS, 75219
Attempt: 367 | Processing record 304 (4 of set 7) | CELEBRATIONS, 4503 LOVERS, 75209
Attempt: 368 | Processing record 305 (5 of set 7) | CENTENNIAL BAR & BUFFET, 300 REUNION, 75202
Attempt: 369 | Processing record 306 (6 of set 7) | CENTRAL CAFE & GRILL, 10000 CENTRAL, 75231
Attempt: 370 | Processing record 307 (7 of set 7) | CESAR TACOS, 2450 COCKRELL HILL, 75211
Attempt: 371 | Processing record 308 (8 of set 7) | CESAR'S TACOS, 2019 NORTHWEST, 75220
Attempt: 372 | Processing record 309 (9 of set 7) | CESAR'S TACOS, 2919 DAVIS, 75211
Attempt: 373 | Processing record 310 (10 of set 7) | CESAR'S TACOS, 1106 DAVIS, 75208
Attempt: 374 | Processing record 311 (11 of set 7) | CESAR'S TACOS, 3025 WESTMORELAND, 75233
Attempt: 375 | Processing record 312 (12 of set 7) | CEVICHE, 451 DAVIS, 75208
Skipping...  A

...

Attempt: 2492 | Processing record 2020 (20 of set 41) | THE CAPITAL GRILLE, 500 CRESCENT, 75201
Attempt: 2493 | Processing record 2021 (21 of set 41) | THE CARIBBEAN CABANA, 920 HARWOOD, 75201
Attempt: 2494 | Processing record 2022 (22 of set 41) | THE CLOVER CLUB, 2404 CEDAR SPRINGS, 75201
Attempt: 2495 | Processing record 2023 (23 of set 41) | THE COMMISSARY, 2201 TUCKER, 75214
Attempt: 2496 | Processing record 2024 (24 of set 41) | THE COMMON TABLE, 2917 FAIRMOUNT, 75201
Attempt: 2497 | Processing record 2025 (25 of set 41) | THE CONTINENTAL RESTAURANT, 3011 GULDEN, 75212
Attempt: 2498 | Processing record 2026 (26 of set 41) | THE CRAB STATION, 2813 COMMERCE, 75226
Attempt: 2499 | Processing record 2027 (27 of set 41) | THE CRAB STATION OYSTER BAR, 9780 WALNUT, 75243
Attempt: 2500 | Processing record 2028 (28 of set 41) | THE DAPPER DOUGHNUT, 13350 DALLAS, 75240
Attempt: 2501 | Processing record 2029 (29 of set 41) | THE DEN & RUMOR LOUNGE, 2710 MCKINNEY, 75204
Attempt: 2502 | Proce

Skipping...  Attempt: 2580 | Record (TILLMANS ROADHOUSE, 324 7TH, 75208) is not found.
Attempt: 2581 | Processing record 2094 (44 of set 42) | TIME OUT TAVERN, 5101 LOVERS, 75209
Attempt: 2582 | Processing record 2095 (45 of set 42) | TIN ROOM, 2514 HUDNALL, 75235
Attempt: 2583 | Processing record 2096 (46 of set 42) | TINY VICTORIES, 604 TYLER, 75208
Attempt: 2584 | Processing record 2097 (47 of set 42) | TIPICO'S, 3118 NORTHWEST, 75220
Attempt: 2585 | Processing record 2098 (48 of set 42) | TIPPY'S THAI CAFE, 12835 PRESTON, 75230
Attempt: 2586 | Processing record 2099 (49 of set 42) | TIPPY'S THAI CAFE, 12835 PRESTON, 75230
Attempt: 2587 | Processing record 2100 (50 of set 42) | TJ CAJUN SEAFOOD & WINGS, 8131 FOREST, 75243
Attempt: 2588 | Processing record 2101 (1 of set 43) | TJ'S FRESH SEAFOOD MARKET & GRILL, 6025 ROYAL, 75220
Attempt: 2589 | Processing record 2102 (2 of set 43) | TOASTED COFFEE & KITCHEN, 5420 ROSS, 75206
Skipping...  Attempt: 2590 | Record (TOBACCO ZONE, 2017 DAV

Attempt: 2668 | Processing record 2162 (12 of set 44) | UP ON KNOX, 3230 KNOX, 75205
Attempt: 2669 | Processing record 2163 (13 of set 44) | UPTOWN POPCORN, 8687 CENTRAL, 75231
Attempt: 2670 | Processing record 2164 (14 of set 44) | UPTOWN TAVERN & WINE BAR, 3605 MCKINNEY, 75204
Attempt: 2671 | Processing record 2165 (15 of set 44) | UPTOWN THAI MARKET, 4817 ROSS, 75204
Attempt: 2672 | Processing record 2166 (16 of set 44) | UPTOWN URBAN MARKET (KITCHEN), 2600 CEDAR SPRING, 75204
Attempt: 2673 | Processing record 2167 (17 of set 44) | URBAN CAPHE, 6030 LUTHER, 75225
Attempt: 2674 | Processing record 2168 (18 of set 44) | URBAN COFFEE, 901 MAIN, 75202
Attempt: 2675 | Processing record 2169 (19 of set 44) | URBAN TACO, 5321 MOCKINGBIRD, 75206
Skipping...  Attempt: 2676 | Record (URBAN VINES WINE BISTRO, 9219 GARLAND, 75218) is not found.
Attempt: 2677 | Processing record 2170 (20 of set 44) | URBANO CAFE, 1410 FITZHUGH, 75204
Skipping...  Attempt: 2678 | Record (URBANO CATERING, 1410 FIT

Attempt: 2757 | Processing record 2243 (43 of set 45) | WILLIAMS FRIED CHICKEN, 2955 WALNUT HILL, 75229
Attempt: 2758 | Processing record 2244 (44 of set 45) | WILLIAMS FRIED CHICKEN, 238 ILLINOIS, 75224
Attempt: 2759 | Processing record 2245 (45 of set 45) | WILLIAMS FRIED CHICKEN, 2844 BUCKNER, 75228
Attempt: 2760 | Processing record 2246 (46 of set 45) | WILLIAMS FRIED CHICKEN, 2909 ILLINOIS, 75216
Attempt: 2761 | Processing record 2247 (47 of set 45) | WILLIAMS FRIED CHICKEN, 6226 MARVIN D LOVE, 75224
Attempt: 2762 | Processing record 2248 (48 of set 45) | WILLIAMS FRIED CHICKEN, 1212 KIEST, 75224
Attempt: 2763 | Processing record 2249 (49 of set 45) | WILLIAMS FRIED CHICKEN, 3280 FORT WORTH, 75211
Attempt: 2764 | Processing record 2250 (50 of set 45) | WILLIAMS FRIED CHICKEN, 3009 MLK, 75215
Attempt: 2765 | Processing record 2251 (1 of set 46) | WILLIAMS SONOMA, 8687 CENTRAL, 75225
Attempt: 2766 | Processing record 2252 (2 of set 46) | WIMPY'S JR, 1802 SINGLETON, 75212
Attempt: 27

Attempt: 2845 | Processing record 2323 (23 of set 47) | ZOE'S KITCHEN, 3878 OAK LAWN, 75219
Attempt: 2846 | Processing record 2324 (24 of set 47) | ZOE'S KITCHEN, 8018 PARK, 75231
Attempt: 2847 | Processing record 2325 (25 of set 47) | ZOE'S KITCHEN, 6025 ROYAL, 75230
Attempt: 2848 | Processing record 2326 (26 of set 47) | ZOE'S KITCHEN, 5710 LOVERS, 75209
Skipping...  Attempt: 2849 | Record (ZRG CONCEPTS, 2505 FITZHUGH, 75204) is not found.
Skipping...  Attempt: 2850 | Record (ZUMBA BY EVA, 4009 URBAN, 75227) is not found.
----------------------------
Data Retrieval Complete
----------------------------


#### visualize data frame updated with API results

In [15]:
df.head()

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,...,yelp_is_closed,yelp_address_1,yelp_address_2,yelp_zip_code,yelp_name,yelp_price,yelp_rating,yelp_review_count,yelp_category_2,yelp_category_3
0,A & A MART,Routine,5/14/19,81,309,MARSALIS,NaN,75203,309 S MARSALIS AVE,5/1/19,...,False,611 W Jefferson,,75208,Fiesta Mart,$$,2.5,13.0,NaN,NaN
1,A & J BAKERY,Routine,4/30/19,92,3515,OAK LAWN,NaN,75219,3515 OAK LAWN AVE,4/1/19,...,False,3515 Oak Lawn Ave,,75219,A&J Bakery,$$,3.0,55.0,Custom Cakes,Cupcakes
2,A & K MART,Routine,2/7/19,98,5723,SINGLETON,NaN,75212,5723 SINGLETON BLVD,2/1/19,...,False,1521 N Cockrell Hill Rd,,75211,Walmart Supercenter,$,1.5,72.0,NaN,NaN
3,A MOTIONS FOOD MART,Routine,2/22/18,80,829,CORINTH ST,NaN,75203,829 S CORINTH ST RD,2/1/18,...,False,801 S Riverfront Blvd,,75207,Fuel City Tacos,$,4.0,1122.0,NaN,NaN
4,A PLUS GROCERY BEER & WINE,Routine,2/14/19,82,4407,MOCKINGBIRD,NaN,75209,4407 W MOCKINGBIRD LN,2/1/19,...,False,5550 W Lovers Ln,None,75209,Trader Joe's,$$,4.5,63.0,NaN,NaN


#### loop through df: compare Dallas Open Data, Google and Yelp info and assign a match "confidence" score (FuzzyWuzzy WRatio) & visualize df

In [130]:
# loop through df results & complete Dallas Open Data, Google and Yelp info
for index, row in df.iterrows():
    
    # check to see if there is a Google rating for the record, flag it if so    
    if str(row["Rating"]) != "nan":
        df.loc[index, "google_result_flag"] = 1
    else:
        df.loc[index, "google_result_flag"] = 0
    
    # check to see if there is a yelp result for the row, if so continue and flag it
    if str(row["yelp_name"]) != "nan":
        df.loc[index, "yelp_result_flag"] = 1
        
        # assign FuzzyWuzzy WRatio score for Dallas Open Data value vs Yelp, when yelp result/value is available
        dod_name = row["Restaurant Name"]
        y_name = row["yelp_name"]
#         print(f"NAME: {dod_name} vs {y_name} FuzzyWuzzy WRatio: ", fuzz.WRatio(dod_name.lower(), y_name.lower()))
        df.loc[index, "name_match_score"] = fuzz.WRatio(dod_name.lower(), y_name.lower())
        
        dod_street_number = row["Street Number"]
        dod_street_name = row["Street Name"]
        dod_address = f"{dod_street_number} {dod_street_name}"
        y_address = row["yelp_address_1"]
#         print(f"ADDRESS: {dod_address} vs {y_address} FuzzyWuzzy WRatio: ", fuzz.WRatio(dod_address, y_address))
        df.loc[index, "address_match_score"] = fuzz.WRatio(dod_address, y_address)
        
        dod_lat = str(round(float(row["Lat"]),5))
        dod_lng = str(round(float(row["Long"]),5))
        y_lat = str(round(float(row["yelp_lat"]),5))
        y_lng = str(round(float(row["yelp_lng"]),5))
        
        dod_lat_lng = f"({dod_lat}, {dod_lng})"
        y_lat_lng = f"({y_lat}, {y_lng})"
#         print(f"LAT/LNG: {dod_lat_lng} vs {y_lat_lng} FuzzyWuzzy WRatio: ", fuzz.WRatio(dod_lat_lng, y_lat_lng))
        df.loc[index, "lat_lng_match_score"] = fuzz.WRatio(dod_lat_lng, y_lat_lng)
        
    else:
        df.loc[index, "yelp_result_flag"] = 0

df.head()
        

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,...,yelp_price,yelp_rating,yelp_review_count,yelp_category_2,yelp_category_3,google_result_flag,yelp_result_flag,name_match_score,address_match_score,lat_lng_match_score
0,A & A MART,Routine,5/14/19,81,309,MARSALIS,NaN,75203,309 S MARSALIS AVE,5/1/19,...,$$,2.5,13.0,NaN,NaN,1.0,1.0,76.0,22.0,74.0
1,A & J BAKERY,Routine,4/30/19,92,3515,OAK LAWN,NaN,75219,3515 OAK LAWN AVE,4/1/19,...,$$,3.0,55.0,Custom Cakes,Cupcakes,1.0,1.0,95.0,95.0,89.0
2,A & K MART,Routine,2/7/19,98,5723,SINGLETON,NaN,75212,5723 SINGLETON BLVD,2/1/19,...,$,1.5,72.0,NaN,NaN,1.0,1.0,53.0,32.0,63.0
3,A MOTIONS FOOD MART,Routine,2/22/18,80,829,CORINTH ST,NaN,75203,829 S CORINTH ST RD,2/1/18,...,$,4.0,1122.0,NaN,NaN,0.0,1.0,33.0,44.0,68.0
4,A PLUS GROCERY BEER & WINE,Routine,2/14/19,82,4407,MOCKINGBIRD,NaN,75209,4407 W MOCKINGBIRD LN,2/1/19,...,$$,4.5,63.0,NaN,NaN,1.0,1.0,45.0,25.0,63.0


In [131]:
# loop again through df results & complete Dallas Open Data, Google and Yelp info to flag high_match_score_flag
for index, row in df.iterrows():
    
    # check to see & flag if there is a yelp hit and a high confidence match score >= 70 for name and address
    if (row["yelp_result_flag"] == 1) & (row["name_match_score"] >= 80) & (row["address_match_score"] >= 80):
        df.loc[index, "high_match_score_flag"] = 1
    else:
        df.loc[index, "high_match_score_flag"] = 0
        
df.head()
    

,Restaurant Name,Inspection Type,Inspection Date,Inspection Score,Street Number,Street Name,Street Unit,Zip,Street Address,Inspection Month,...,yelp_rating,yelp_review_count,yelp_category_2,yelp_category_3,google_result_flag,yelp_result_flag,name_match_score,address_match_score,lat_lng_match_score,high_match_score_flag
0,A & A MART,Routine,5/14/19,81,309,MARSALIS,NaN,75203,309 S MARSALIS AVE,5/1/19,...,2.5,13.0,NaN,NaN,1.0,1.0,76.0,22.0,74.0,0.0
1,A & J BAKERY,Routine,4/30/19,92,3515,OAK LAWN,NaN,75219,3515 OAK LAWN AVE,4/1/19,...,3.0,55.0,Custom Cakes,Cupcakes,1.0,1.0,95.0,95.0,89.0,1.0
2,A & K MART,Routine,2/7/19,98,5723,SINGLETON,NaN,75212,5723 SINGLETON BLVD,2/1/19,...,1.5,72.0,NaN,NaN,1.0,1.0,53.0,32.0,63.0,0.0
3,A MOTIONS FOOD MART,Routine,2/22/18,80,829,CORINTH ST,NaN,75203,829 S CORINTH ST RD,2/1/18,...,4.0,1122.0,NaN,NaN,0.0,1.0,33.0,44.0,68.0,0.0
4,A PLUS GROCERY BEER & WINE,Routine,2/14/19,82,4407,MOCKINGBIRD,NaN,75209,4407 W MOCKINGBIRD LN,2/1/19,...,4.5,63.0,NaN,NaN,1.0,1.0,45.0,25.0,63.0,0.0


#### organize / reorder columns in df & visualize

In [132]:
df = df.loc[:,["Restaurant Name",
               "yelp_name",
               "name_match_score",
               "Street Number",
               "Street Name",
               "Street Unit",
               "Street Address",
               "Zip",
               "yelp_address_1",
               "yelp_address_2",
               "yelp_zip_code",
               "address_match_score",
               "Lat",
               "Long",
               "yelp_lat",
               "yelp_lng",
               "lat_lng_match_score",
               "Inspection Score",
               "Inspection Type",
               "Inspection Date",
               "Inspection Month",
               "Inspection Year",
               "Rating",
               "Rating Count",
               "Type",
               "yelp_rating",
               "yelp_review_count",
               "yelp_price",
               "yelp_category_1",
               "yelp_category_2",
               "yelp_category_3",
               "yelp_is_closed",
               "google_result_flag",
               "yelp_result_flag",
               "high_match_score_flag"
              ]
           ]
df.head()

,Restaurant Name,yelp_name,name_match_score,Street Number,Street Name,Street Unit,Street Address,Zip,yelp_address_1,yelp_address_2,...,yelp_rating,yelp_review_count,yelp_price,yelp_category_1,yelp_category_2,yelp_category_3,yelp_is_closed,google_result_flag,yelp_result_flag,high_match_score_flag
0,A & A MART,Fiesta Mart,76.0,309,MARSALIS,NaN,309 S MARSALIS AVE,75203,611 W Jefferson,,...,2.5,13.0,$$,Grocery,NaN,NaN,False,1.0,1.0,0.0
1,A & J BAKERY,A&J Bakery,95.0,3515,OAK LAWN,NaN,3515 OAK LAWN AVE,75219,3515 Oak Lawn Ave,,...,3.0,55.0,$$,Bakeries,Custom Cakes,Cupcakes,False,1.0,1.0,1.0
2,A & K MART,Walmart Supercenter,53.0,5723,SINGLETON,NaN,5723 SINGLETON BLVD,75212,1521 N Cockrell Hill Rd,,...,1.5,72.0,$,Department Stores,NaN,NaN,False,1.0,1.0,0.0
3,A MOTIONS FOOD MART,Fuel City Tacos,33.0,829,CORINTH ST,NaN,829 S CORINTH ST RD,75203,801 S Riverfront Blvd,,...,4.0,1122.0,$,Convenience Stores,NaN,NaN,False,0.0,1.0,0.0
4,A PLUS GROCERY BEER & WINE,Trader Joe's,45.0,4407,MOCKINGBIRD,NaN,4407 W MOCKINGBIRD LN,75209,5550 W Lovers Ln,None,...,4.5,63.0,$$,Grocery,NaN,NaN,False,1.0,1.0,0.0


#### export restaurant_df to output CSV data file

In [133]:
# df.to_csv(f"{output_data_filepath}yelp_output.csv", encoding="utf-8", index=False)
df.to_csv(f"{output_data_filepath}yelp_output.csv", encoding="UTF-16LE", index=False)